In [13]:
import math
import re
import heapq
import csv

import pandas as pd
import numpy as np

import time

In [14]:
import numpy.linalg as LA

from IPython.display import clear_output
from fingerprint.fingerprint import get_fingerprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#### Before you start, make sure you add 'unique_sorted_split.txt' to the data folder and create the output folder


## Settings

In [19]:
input_filename = 'output/labels_clusters.csv'#'data/unique_sorted_split.txt' 

mapping_filename = 'mapping_test.csv'#'output/mapping_clusters.csv'
label_filename = 'label_test.csv'#'output/labels_clusters.csv'

threshold = 0.85


## Load data

In [20]:
df = pd.read_csv(input_filename, names=["ID", "Organization"], quoting=csv.QUOTE_NONE)


## Run Tf Idf Clustering

In [21]:
id_table = {}
ID = 0

In [22]:
id_table_file = open(label_filename, "w")
mapping = open(mapping_filename, "w")

vectorizer = TfidfVectorizer()

label = get_fingerprint(df.ix[0].Organization)

corpus = set()
corpus.update(label.split())

start = time.time()

for i, s in df.iterrows():
    if i % 10000 == 0 :
        clear_output()
        elapsed = time.time()-start
        print(f"Latest label: {label}")
        print(f"{i} / {df.shape[0]}\nElapsed: {elapsed}")
        print(f"Estimated timeleft: {df.shape[0]/float(i+1)*elapsed/60/60}h")
        
    original_string = s.Organization
    cleaned_string = get_fingerprint(original_string)
    cleaned_split = cleaned_string.split()
    
    if len(cleaned_split) < 2:
        mapping.write(f"{original_string},-1\n")
        continue

    train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
    test = vectorizer.transform([cleaned_string]).toarray()
    score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]

    if np.isnan(score) or score < threshold: # Create new cluster
        corpus = set()
        corpus.update(cleaned_split)
        train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
        test = vectorizer.transform([cleaned_string]).toarray()
        score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]
        label = cleaned_string
    
    if label not in id_table: # Create new cluster group if not exists
        ID += 1        
        id_table[label] = ID
        id_table_file.write(f"{ID},{label}\n")
        
    label_id = id_table[label]
    mapping.write(f"{original_string},{label_id}\n")

labels.close()
mapping.close()

Latest label: department of hospitalist
10000 / 47621
Elapsed: 37.36250162124634
Estimated timeleft: 0.04941838287575054h


KeyboardInterrupt: 